# Unweighted Graphs:


https://learning.ecam.be/SA4T/slides/04-graphs

A graph is a set of vertices and edges. An edge join two vertices.

<img src="img/graphs.png" width="35%">

- Directed Graphs : edges have directions
- Acyclic : no loops
- Weighted : edges have weights (i.e. a certain `"cost"`)

### Adjacency List :

An adjacency list is a dictionary whose keys are vertices, and the associated values are a list (or set) of the end points of the edges starting from that
vertex.

In [28]:
# a has an edge to "b" and "c"
# b has an edge to "c", "d" and "a"  #! undirected graph : a <-> b

adj = {
    "a": {"b", "c"},
    "b": {"c", "d", "a"},
    "c": {"d", "e"},
    "d": set(),
    "e": set(),
}

## Application in software Development : Reactivity


We will show how `state` and `effect` on the next slide.
- `State:` variable that changes over time and that is usually has an impact on the UI.
- `Effect:` function that is run whenever an underlying signal changes. Generally used to update the UI.

_Information:_
- All web frameworks (Angular, Vue, Svelte, Solid, Qwik, etc.) except React use this pattern at the core of their reactivity system.
In a way, this is "Excel programming"

### Demo : Svelte

Use state variables to change the UI. The following code should give 1 progress bar and 2 buttons. Showing the healthof pikachu and buttons to slap or heal Pikachu.

<script>
  let maxHp = 274
  let hp = $state(274)

  function slap() {
    hp -= 10
  }
  function heal() {
    hp = maxHp
  }
</script>

Pikachu<br />
<progress value={hp} max={maxHp} /><br />
{hp} / {maxHp}<br />
<button onclick={slap}>Slap Pikachu</button>
<button onclick={heal}>Heal Pikachu</button>
{#if hp <= 0}
  <p>You've killed Pikachu. What a monster!</p>
{/if}

#### Herhaling Decorators

- Definition : `@decorator` above def f(...): is the same as `f = decorator(f)` — the function object f is passed to decorator and the decorator's return value replaces the original name.



- 

In [11]:
# Before / After Functions behavior

def deco(fn):
    def wrapped(*args, **kwargs):
        print("before")
        result = fn(*args, **kwargs)
        print("after")
        return result
    return wrapped

@deco
def greet(name):
    return "Hello " + name

greet("Alice")

before
after


'Hello Alice'

In [10]:
# Repeat Decorators (function calls)

def repeat(n):
    def decorator(fn):
        def wrapped(*args, **kwargs):
            for _ in range(n):
                fn(*args, **kwargs)
        return wrapped
    return decorator

@repeat(3)
def ping(): print("ping")

ping()

ping
ping
ping


In [ ]:
# Class-based Decorator (ex. count executions)
class CountCalls:
    def __init__(self, fn):
        self.fn = fn
        self.count = 0
    def __call__(self, *args, **kwargs):
        self.count += 1
        return self.fn(*args, **kwargs)

@CountCalls
def f(x): return x*2


f(2)
assert(f.count == 1)
f(4)
assert(f.count == )
# f(56)


SyntaxError: invalid syntax (1646817172.py, line 15)

In [ ]:
# Preserving Metadata
from functools import wraps
def deco(fn):
    @wraps(fn)
    def wrapped(*a, **k):
        return fn(*a, **k)
    return wrapped

# @property makes a method behave like a read-only attribute: 
    # obj.value calls the getter.
# @value.setter defines the setter for that property name; 
    # together they provide attribute accessors without changing call sites.



### State Implementation :

- `running`: stack used to keep track of currently running effects. Later, we will
ensure that effects push themselves onto running before running, and remove themselves afterwards.

- `self._value`: used to store the current value of the signal.
- `self.subscribers`: set containing the effects that need to be rerun. In a way, this is an adjacency set.


- L9-13: signal getter. If the signal is read inside an effect, we add the latter in `self.subscribers`.
- L15-19: signal setter. We change the value and rerun the dependent effects.
- L22-28: effect decorator. We change fn so that it pushes itself on and off the `running stack.

In [ ]:
running = []
class State:
    def __init__(self, value):
        self._value = value
        self.subscribers = set()

    @property                       # @property gives a getter for .value
    def value(self):
        if running:
            self.subscribers.add(running[-1])
        return self._value

    @value.setter
    def value(self, value):
        self._value = value
        for effect in self.subscribers:
            effect()

def effect(fn):
    """Show running functions in the running list."""
    def wrapped():
        running.append(wrapped)
        fn()
        running.pop()
    wrapped()

In [4]:
hp = State(100)
doubled = Derived(lambda: hp.value * 2)

@effect
def on_hp_change():
    if hp.value > 20:
        print("You have", hp.value, "HP")
    else:
        print("Careful! Only", hp.value, "HP left")
    print("Doubled value:", doubled.value)


hp.value = 90
hp.value = 70
hp.value = 15

NameError: name 'Derived' is not defined

## Graph Exploration:

<img src="img/bfs_dfs.png" width="100%">

### BFS :


ex. Shortest-Path algorithms


In [ ]:
# Go through all vertices ? why


def BFS(adj, start):
    visited = set()                 # already visited nodes/
    queue = [start]                 # FIFO queue of next nodes to be visited
    while queue:
        node = queue.pop(0)         # dequeue is better
        print("Visiting", node)
        visited.add(node)
        queue.extend(adj[node] - visited) # To visit nex : adjacent neighbors of node - already visited nodes
# O(V + E + n) :
    # while : O(V)
    # .extend() : O(E)
    # .pop(0) : O(n)


# queue.pop(0) shifts all elements : O(n)


# using collections
import collections
def BFS_2(adj, start):
    visited = set()
    queue = collections.deque([start])  # deque: a data structure optimized for fast FIFO operation

    while queue:
        node = queue.popleft()            # O(1)
        print("Visiting: ", node)         #
        visited.add(node)                 # O(1)
        queue.extend(adj[node] - visited) # add neighbor adjacent nodes to visit next
# O(V + E) :
    # while : O(V)
    # .extend() : O(E)


import collections
def BFS_3(adj, start):
    visited = {start}
    queue = collections.deque([start])  # deque: a data structure optimized for fast FIFO operation

    while queue:
        node = queue.popleft()          # O(1)
        print("Visiting: ", node)
        for neighbor in adj.get(node,()):
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor) # add neighbor adjacent nodes to visit next
# O(V) :
    # while : O(V)




BFS(
    { 
        0: {1, 2}, 
        1: {3, 4}, 
        2: {5, 6}, 
        3: set(), 4: set(), 5: set(), 6: set() 
    }, 
        0
    )


# Complexity : O(V + E)\
    # O(V) : (while) due to dequeing each Vertex
    # O(E) : (queue.ext) due to examining each neighbor of each Vertex
    # O(2E) : for undirected graphs.
    # Why not O(V*E) ? -> we don't re-process each Vertex for each neighbor (thanks to visited)

#! Exam : explain O(V + E)



Visiting:  0
Visiting:  1
Visiting:  2
Visiting:  3
Visiting:  4
Visiting:  5
Visiting:  6


### DFS :

- One-solution algorithm. 

- Backtracking.

ex. Sudoku

<img src="img/tree.png" width="35%">

In [2]:


def DFS(adj: dict[any, set]):
    visited = set()


    def explore(start):
        visited.add(start)
        print(f"Visiting: {start}")
        for neighbor in adj[start] - visited:
            explore(neighbor)
        print(f"Finish exploring: {start}")

    # explore(0)   # 0, 1, 3, 4, 2, 5, 6: 

    for node in adj:
        if node not in visited:
            explore(node)

DFS(
    { 
        0: {1, 2}, 
        1: {3, 4}, 
        2: {5, 6}, 
        3: set(), 4: set(), 5: set(), 6: set() 
    })






Visiting: 0
Visiting: 1
Visiting: 3
Finish exploring: 3
Visiting: 4
Finish exploring: 4
Finish exploring: 1
Visiting: 2
Visiting: 5
Finish exploring: 5
Visiting: 6
Finish exploring: 6
Finish exploring: 2
Finish exploring: 0


### X1 : modify BFS to get shortest Path

In [3]:
# copy bfs and add a dict to track

import collections

def shortest_paths(adj, start):
    dist = {start: 0}           # store the shortest-known distance to a node, init with start
    # visited = set()  # can remove ?
    queue = collections.deque([start])
    
    while queue:
        node = queue.popleft()
        print("Visiting: ", node)
        # visited.add(node)
        for neighbor in adj[node] - set(dist.keys()):   # visit each neighbor in adj - already visited nodes
            if neighbor not in dist:                    # redundant
                dist[neighbor] = dist[node] + 1
                queue.append(neighbor)
    print(dist)

shortest_paths(
    { 
        0: {1, 2}, 
        1: {3, 4}, 
        2: {5, 6}, 
        3: set(), 4: set(), 5: set(), 6: set() 
    }, 
        0
    )

# step by step:
    # queue = [0]
    # node = 0, queue = []
    # visited = {0}
    # queue = [1,2]    # whats the distance to 1 and 2 ?




Visiting:  0
Visiting:  1
Visiting:  2
Visiting:  3
Visiting:  4
Visiting:  5
Visiting:  6
{0: 0, 1: 1, 2: 1, 3: 2, 4: 2, 5: 2, 6: 2}


### x2 : Sudoku

- A grid will be represented by an array of 81 integers between `0` and `9` We use `0` to indicate that the entry is empty.
- An edge exists between two grids if they differ by only one number, and they both satisfy the sudoku rules (they are not necessarily solvable). 
- Explore the graph until you get a solve grid. If it's impossible, `backtrack`.


- DFS : because single solution, dont want to explore all possibilities

In [5]:

# def blacklist(grid: list[int], n: int) -> set[int]:
#     """ 
#         Return numbers that cannot be placed at position n
#         (because they already appear in same row, column, or 3x3 box)
#         For entry n,
#         specify whuch numbers cannot be used,
#         bc they have been used in the row, column 
#         grid = [1, 0, 0,...] => blacklist(grid, 1) = 1
#     """
#     blacklisted = set()
#     # Find set of numbers already in the row, add them to res
#     row_number = grid[n//9]
#     row_vals = {i for i in grid[row_number * 9 + 9] if v != 0}

#     # Find // columns
#     column_number = grid[n%9]
#     column_vals = {grid[column_number + 9*i] for i in range(9) if i != 0}

#     # Find // in 3x3 box (choose top-left as reference):
#     box_vals = set()
#     box_row = (row_number //3) * 3
#     box_col = (column_number // 3) * 3
#     for next_right in (0,1,2) : # get next 3 right and next 3 down
#         for next_down in (0,1,2):
#             val_id =  (box_row + next_right) * 9 + (box_column + next_down)
#             val = grid[val_id]
#             if val != 0: box_vals.add(val)

#     return row_vals | column_vals | box_vals

def pretty_print(grid):
    """Print a flat 81-element sudoku `grid` as a 9x9 board.
    Empty cells (0) are printed as '.'.
    """
    if len(grid) != 81:
        raise ValueError("grid must be length 81")
    for r in range(9):
        # build row as strings (use '.' for empty)
        row = [(str(grid[r*9 + c]) if grid[r*9 + c] != 0 else '.') for c in range(9)]
        # join with vertical separators for 3x3 blocks
        line = ' '.join(row[0:3]) + ' | ' + ' '.join(row[3:6]) + ' | ' + ' '.join(row[6:9])
        print(line)
        # print horizontal separator after every 3 rows (except after last)
        if r % 3 == 2 and r != 8:
            print('-' * 21)

def blacklist_prof(grid: list[int], n: int) -> set[int]:
    i, j = n // 9, n % 9
    # row values (indices 9*i .. 9*i+8)
    row = {grid[9*i + k] for k in range(9)}
    # column values (indices j, j+9, j+18, ...)
    col = {grid[9*k + j] for k in range(9)}
    # 3x3 region: top-left corner of the region
    x, y = (i // 3) * 3, (j // 3) * 3
    region = { grid[9*(x + dx) + (y + dy)] for dx in range(3) for dy in range(3) }
    # union, remove 0 (empty cells)
    return (row | col | region) - {0}


def solve(grid: list[int]):
    """ Explore depth first
        if it worked, return grid
        if not, backtrack
    """
    # Find first 0
    # n = 0
    # for i in grid:
    #     if grid[i] == 0: n = i
    #     if i ==len(grid): return grid  # solved no more 0
    if 0 not in grid : return grid
    n = grid.index(0)
    # Loop over all neighboring grids (same Row)
    for i in range(1, 10):
        if i not in blacklist_prof(grid, n):
            grid[n] = i               # place candidate
            result = solve(grid)      # recurse
            if 0 not in result :      # success, bubble up
                return result         #

    grid[n] = 0               #Failure, backtrack
    return grid


def solve_final(grid: list[int]):
    """ Explore depth first
        if it worked, return grid
        if not, backtrack
    """
    # Find first 0
    if 0 not in grid : 
        return grid
    n = grid.index(0)
    # Loop over all neighboring grids (same Row)
    for i in set(range(1, 10)) - blacklist_prof(grid, n):
        grid[n] = i               # place candidate
        result = solve(grid)      # recurse
        if 0 not in result :      # success, bubble up
            return result         #
        grid[n] = 0               #Failure, backtrack
    return grid

# ! tab for grid[n] = 0 ??


pretty_print(solve_final(81*[0]))

# 1 ere Row : [ 1, 2, 3, 4, 0, 6, 7, 8, 9 ] --> 5
# 1 ere Row : [ 1, 2, 3, 4, 5, 6, 7, 8, 9 ]
# 2 eme Row : 



1 2 3 | 4 5 6 | 7 8 9
4 5 6 | 7 8 9 | 1 2 3
7 8 9 | 1 2 3 | 4 5 6
---------------------
2 1 4 | 3 6 5 | 8 9 7
3 6 5 | 8 9 7 | 2 1 4
8 9 7 | 2 1 4 | 3 6 5
---------------------
5 3 1 | 6 4 2 | 9 7 8
6 4 2 | 9 7 8 | 5 3 1
9 7 8 | 5 3 1 | 6 4 2


### Snake & Ladders :

Snakes and Ladders is a game played on a 10 x 10 board, the goal of which is get from square 1 to square 100. On each turn players will roll a six-sided die and move forward a number of spaces equal to the result. 
- If they land on a square that represents a snake or ladder, they will be transported ahead or behind, respectively, to a new square.
- Find the smallest number of turns it takes to win.
- Part II: find the path itself.

In [ ]:



snakes = {17: 13, 52: 29, 57: 40, 62: 22, 88: 18, 95: 51, 97: 79}
ladders = {3: 21, 8: 30, 28: 84, 58: 77, 75: 86, 80: 100, 90: 91} 


def build_adj(snake, ladders):
    adj = {}
    for n in range(101):
        adj[n] = set()
        for roll in range(6):
            # Base Case



def minimum_turns(snakes, ladders):
    queue = collections.deque([start])
    ist = {start: 0}
    # find adj moves in snakes and ladder
    adj = [snakes[start]]
    adj.append(ladders[start])

    while queue:
        node = queue.popleft()
        print("Visiting: ", node)
        for d i range(1, 7):     # list possible DICE moves
            next = node + d
            if next in ladders:

    return 0




# previous code reference
def shortest_paths(adj, start):
    dist = {start: 0}
    # visited = set()  # can remove ?
    queue = collections.deque([start])
    
    while queue:
        node = queue.popleft()
        print("Visiting: ", node)
        # visited.add(node)
        for neighbor in adj[node] - set(dist.keys()):
            if neighbor not in dist:
                dist[neighbor] = dist[node] + 1
                queue.append(neighbor)
    print(dist)



 
def minimum_turns_PROF(snakes, ladders):
    visited = set()
    board = { n: n for n in range(0, 101) }
    for start, end in (snakes | ladders).items():
        board[start] = end
    queue = [(0, 0)]
    while queue:
        square, turns = queue.pop(0)
        for move in range(square + 1, square + 7):
            move = board[move]
            if move >= 100:
                return turns + 1
            if move not in visited:
                visited.add(move)
                queue.append((move, turns + 1))

minimum_turns(snakes, ladders)


 
def minimum_turns_BRUNO(snakes, ladders):
    adj = {i: set() for i in range(1, 101)}
    for i in range(1, 100):
        for dice in range(1, 7):
            j = i + dice
            if j > 100:
                continue
            if j in snakes:
                j = snakes[j]
            elif j in ladders:
                j = ladders[j]
            adj[i].add(j)
    dist = shortest_paths(adj, 1)
    if 100 in dist:
        return dist[100]

print(minimum_turns(snakes, ladders))






IndentationError: expected an indented block after 'for' statement on line 9 (1987244501.py, line 14)

## Topological Sort :

Given a directed graph, "sort" the vertices in such a way that the edges are all pointing right.
- Topological sort: wake up, brush teeth, shower, get dressed, eat, go to school.

- Run DFS on a graph you can see and log at the beginning and at the end of an "exploration".

<img src="img/topo.png" width="100%">

In [26]:

#? models “must happen before” constraints: task scheduling, build systems, course prerequisites, etc.

# DFS : wake up -> brush teeth -> goto school
# look at when DFS finishes
# invert order

# WU - shower -> get dresed -> school ->



def topological_sort(adj: dict[any, set]) -> list:
    visited = set()
    ordered = []

    def explore(start):
        visited.add(start)                      # mark as visited
        print(f"Visiting: {start}")             # 
        for neighbor in adj[start] - visited:   # 
            explore(neighbor)
        print(f"Finish exploring: {start}")
        ordered.append(start)

    for node in adj:
        if node not in visited:
            explore(node)

    return reversed(ordered)



# Wake up = 1
# eat = 2
# Brush teeth = 3
# shower = 4
# get dressed = 5
# goto school = 6

routine  = { 
    1: {2,3,4,5,6},
    2 : {3, 6},
    3 : {6},
    4 : {5, 6},
    5 : {6},
    6 : set()
}


# topological_sort({ 1: {2}, 0: {1}, 2: {3}, 3: set() })
topological_sort(routine)





Visiting: 1
Visiting: 2
Visiting: 3
Visiting: 6
Finish exploring: 6
Finish exploring: 3
Visiting: 6
Finish exploring: 6
Finish exploring: 2
Visiting: 3
Finish exploring: 3
Visiting: 4
Visiting: 5
Finish exploring: 5
Finish exploring: 4
Visiting: 5
Finish exploring: 5
Visiting: 6
Finish exploring: 6
Finish exploring: 1


[]

### Detecting cycles in undirected graphs :

Given an undirected graphs, determine if it has a cycle (loop).

In [ ]:
# explore graph
# check if already explored
# BFS, DFS are equivalent ?

def has_cycles(adj: dict[any, set]) -> bool:
    return False

has_cycle  = { 
    1: {2,3,4,5,6},
    2 : {3, 6},
    3 : {6},
    4 : {5, 6},
    5 : {6, 1},    # points back to 1 : cycle
    6 : set()
}

no_cycle  = { 
    1: {2,3,4,5,6},
    2 : {3, 6},
    3 : {6},
    4 : {5, 6},
    5 : {6},
    6 : set()
}

has_cycles(has_cycle)   # True
has_cycles(no_cycle)    # False